In [47]:
import argparse
from argparse import RawTextHelpFormatter
import os
import pickle
import random
import numpy as np
import torch
from torch_geometric.transforms import BaseTransform
from torch_geometric.utils.num_nodes import maybe_num_nodes 
from torch_geometric.loader import DataLoader
from math import ceil
import torch.nn.functional as F
from torch_geometric.nn import GINConv,GCNConv, MLP, DenseGINConv,DenseGCNConv, PANConv,dense_mincut_pool, dense_diff_pool, DMoNPooling,global_add_pool,TopKPooling, PANPooling, SAGPooling, ASAPooling, EdgePooling,graclus,MemPooling
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.utils import to_dense_adj, to_dense_batch,add_self_loops, scatter,degree, to_undirected,softmax
from typing import Callable, Optional, Tuple, Union,Any
from torch_geometric.data import Batch, Data,InMemoryDataset
from torch_geometric.nn.pool.consecutive import consecutive_cluster
from torch_geometric.nn.pool.pool import pool_batch, pool_edge, pool_pos
from torch_geometric.nn.pool.topk_pool import topk, filter_adj
from torch.nn import Module,Linear
from torch_scatter import scatter, scatter_add, scatter_min ,scatter_max
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor

In [2]:
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]

In [3]:
rng = np.random.default_rng(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
parser = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter)
parser.add_argument('--pooling', type=str, default='comp-graclus',
                    help="Options:\n None (no-pool)\n 'diffpool'\n 'mincut'\n" 
                    " 'dmon'\n 'edgepool'\n 'graclus'\n 'kmis'\n 'topk'\n 'panpool'\n"
                    " 'asapool'\n 'sagpool'\n 'dense-random'\n 'sparse-random'\n"
                    " 'comp-graclus'\n")
parser.add_argument('--pool_ratio', type=float, default=0.1)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--hidden_channels', type=int, default=64)
parser.add_argument('--num_layers_pre', type=int, default=2)
parser.add_argument('--num_layers_post', type=int, default=1)
parser.add_argument('--lr', type=float, default=1e-4)
parser.add_argument('--epochs', type=int, default=500)
parser.add_argument('--runs', type=int, default=1)

args, unknown_args = parser.parse_known_args()

print(args)

Namespace(pooling='comp-graclus', pool_ratio=0.1, batch_size=32, hidden_channels=64, num_layers_pre=2, num_layers_post=1, lr=0.0001, epochs=500, runs=1)


In [5]:
class DataToFloat(BaseTransform):
    def __call__(self, data):
        data.x = data.x.to(torch.float32)
        return data

In [6]:
class EXPWL1Dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super(EXPWL1Dataset, self).__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["EXPWL1.pkl"]

    @property
    def processed_file_names(self):
        return 'data.pt' 

    def process(self):
        # Read data into huge `Data` list.
        data_list = pickle.load(open(os.path.join(self.root, "raw/EXPWL1.pkl"), "rb"))

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [7]:
path = "EXPWL1/"
dataset = EXPWL1Dataset(path, transform=DataToFloat()) 

In [8]:
# compute avg number of nodes
avg_nodes = int(dataset.data.num_nodes/len(dataset))

c:\Users\USER\anaconda3\envs\un\lib\site-packages\torch_geometric\data\in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [9]:
# compute max number of nodes
max_nodes = 0
for d in dataset:
    max_nodes = max(d.num_nodes, max_nodes)
if args.pooling == 'sparse-random':
    max_nodes *= args.batch_size

In [10]:
def negative_edges(edge_index, num_nodes=None, num_neg_samples=None,
                      force_undirected=False):

    num_nodes = maybe_num_nodes(edge_index, num_nodes)

    # Handle '|V|^2 - |E| < |E|' case for G = (V, E).
    num_neg_samples = num_nodes * num_nodes - edge_index.size(1)

    if force_undirected:
        num_neg_samples = num_neg_samples // 2

        # Upper triangle indices: N + ... + 1 = N (N + 1) / 2
        rng = range((num_nodes * (num_nodes + 1)) // 2)

        # Remove edges in the lower triangle matrix.
        row, col = edge_index
        mask = row <= col
        row, col = row[mask], col[mask]

        # idx = N * i + j - i * (i+1) / 2
        idx = (row * num_nodes + col - row * (row + 1) // 2).to('cpu')
    else:
        rng = range(num_nodes**2)
        # idx = N * i + j
        idx = (edge_index[0] * num_nodes + edge_index[1]).to('cpu')

    perm = torch.tensor(rng)
    mask = torch.from_numpy(np.isin(perm, idx)).to(torch.bool)
    rest = mask.nonzero().view(-1)
    while rest.numel() > 0:  # pragma: no cover
        tmp = torch.tensor(random.sample(rng, rest.size(0)))
        mask = torch.from_numpy(np.isin(tmp, idx)).to(torch.bool)
        perm[rest] = tmp
        rest = rest[mask.nonzero().view(-1)]

    if force_undirected:
        # (-sqrt((2 * N + 1)^2 - 8 * perm) + 2 * N + 1) / 2
        row = torch.floor((-torch.sqrt((2. * num_nodes + 1.)**2 - 8. * perm) +
                           2 * num_nodes + 1) / 2)
        col = perm - row * (2 * num_nodes - row - 1) // 2
        neg_edge_index = torch.stack([row, col], dim=0).long()
        neg_edge_index = to_undirected(neg_edge_index)
    else:
        row = perm / num_nodes
        col = perm % num_nodes
        neg_edge_index = torch.stack([row, col], dim=0).long()

    return neg_edge_index.to(edge_index.device)

In [11]:
def batched_negative_edges(edge_index, batch, num_neg_samples=None,
                              force_undirected=False):
  
    split = degree(batch[edge_index[0]], dtype=torch.long).tolist()
    edge_indices = torch.split(edge_index, split, dim=1)
    num_nodes = degree(batch, dtype=torch.long)
    cum_nodes = torch.cat([batch.new_zeros(1), num_nodes.cumsum(dim=0)[:-1]])

    neg_edge_indices = []
    for edge_index, N, C in zip(edge_indices, num_nodes.tolist(),
                                cum_nodes.tolist()):
        neg_edge_index = negative_edges(edge_index - C, N, num_neg_samples,
                                           force_undirected) + C
        neg_edge_indices.append(neg_edge_index)

    return torch.cat(neg_edge_indices, dim=1)

In [12]:
def _sum_pool_x(
    cluster: Tensor,
    x: Tensor,
    size: Optional[int] = None,
) -> Tensor:
    return scatter(x, cluster, dim=0, dim_size=size, reduce='add')


In [13]:


def sum_pool_x(
    cluster: Tensor,
    x: Tensor,
    batch: Tensor,
    size: Optional[int] = None,
) -> Tuple[Tensor, Optional[Tensor]]:

    if size is not None:
        batch_size = int(batch.max().item()) + 1
        return _sum_pool_x(cluster, x, batch_size * size), None

    cluster, perm = consecutive_cluster(cluster)
    x = _sum_pool_x(cluster, x)
    batch = pool_batch(perm, batch)

    return x, batch


In [14]:
def sum_pool(cluster: Tensor,data: Data,transform: Optional[Callable] = None,) -> Data:

    cluster, perm = consecutive_cluster(cluster)

    x = None if data.x is None else _sum_pool_x(cluster, data.x)
    index, attr = pool_edge(cluster, data.edge_index, data.edge_attr)
    batch = None if data.batch is None else pool_batch(perm, data.batch)
    pos = None if data.pos is None else pool_pos(cluster, data.pos)

    data = Batch(batch=batch, x=x, edge_index=index, edge_attr=attr, pos=pos)

    if transform is not None:
        data = transform(data)

    return data

In [15]:
class RndSparse(torch.nn.Module):
    def __init__(
        self,
        ratio: Union[int, float] = 0.5,
        max_nodes=None
    ):
        super().__init__()

        self.ratio = ratio
        self.min_score = None
        self.score = torch.randn(max_nodes)


    def forward(
        self,
        x: Tensor,
        edge_index: Tensor,
        edge_attr: Optional[Tensor] = None,
        batch: Optional[Tensor] = None,
    ) -> Tuple[Tensor, Tensor, Optional[Tensor], Tensor, Tensor, Tensor]:

        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        
        score = softmax(self.score[:x.size(0)].to(x.device), batch)

        perm = topk(score, self.ratio, batch, self.min_score)
        x = x[perm] * score[perm].view(-1, 1)
        batch = batch[perm]
        edge_index, edge_attr = filter_adj(edge_index, edge_attr, perm,
                                           num_nodes=score.size(0))

        return x, edge_index, edge_attr, batch, perm, score[perm]

In [16]:
def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()

    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  # self-loops

        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()

        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  # self-loops

        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n

    return mis

In [17]:
def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device

    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis

    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)

    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]

In [18]:
class KMISPooling(Module):

    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }

    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."

        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."

        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough

        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops

        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"

            self.lin = Linear(in_channels=in_channels, out_channels=1,
                              weight_initializer='uniform')

    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x

        row, col, _ = adj.coo()
        x = x.view(-1)

        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()

        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)

        return x / k_sums

    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()

        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)

        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)

        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)

        if self.scorer == 'first':
            return x[..., [0]]

        if self.scorer == 'last':
            return x[..., [-1]]

        return self.scorer(x, edge_index, edge_attr, batch)

    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        adj, n = edge_index, x.size(0)

        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))

        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)
        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)

        row, col, val = adj.coo()
        c = mis.sum()

        if val is None:
            val = torch.ones_like(row, dtype=torch.float)

        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)

        if self.remove_self_loops:
            adj = remove_diag(adj)

        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)

        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)

        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])

        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None
            
        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])

        if batch is not None:
            batch = batch[mis]
            

        return x, edge_index, edge_attr, batch, mis, cluster

    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""

        return f'{self.__class__.__name__}({channels}k={self.k})'

In [44]:
class GIN_Pool_Net(torch.nn.Module):
    def __init__(self, 
                 in_channels,           # Size of node features
                 out_channels,          # Number of classes
                 num_layers_pre=1,      # Number of GIN layers before pooling
                 num_layers_post=1,     # Number of GIN layers after pooling
                 hidden_channels=64,    # Dimensionality of node embeddings
                 norm=True,             # Normalise Layers in the GIN MLP
                 activation='ELU',      # Activation of the MLP in GIN 
                 average_nodes=None,    # Needed for dense pooling methods
                 max_nodes=None,        # Needed for random pool
                 pooling=None,          # Pooling method
                 pool_ratio=0.1,        # Ratio = nodes_after_pool/nodes_before_pool
                 ):
        super(GIN_Pool_Net, self).__init__()
        
        self.num_layers_pre = num_layers_pre
        self.num_layers_post = num_layers_post
        self.hidden_channels = hidden_channels
        self.act = activation_resolver(activation)
        self.pooling = pooling
        self.pool_ratio = pool_ratio
  
        # Pre-pooling block            
        self.conv_layers_pre = torch.nn.ModuleList()
        if pooling=='panpool':
            for _ in range(num_layers_pre):
                self.conv_layers_pre.append(PANConv(in_channels, hidden_channels, filter_size=2))
                in_channels = hidden_channels
        else:
            for _ in range(num_layers_pre):
                mlp = MLP([in_channels, hidden_channels, hidden_channels], act=activation)
                self.conv_layers_pre.append(GINConv(nn=mlp, train_eps=False))
                in_channels = hidden_channels
                        
        # Pooling block
        pooled_nodes = ceil(pool_ratio * average_nodes)
        if pooling in ['diffpool','mincut']:
            self.pool = Linear(hidden_channels, pooled_nodes)
        elif pooling=='dmon':
            self.pool = DMoNPooling(hidden_channels, pooled_nodes)
        elif pooling=='dense-random':
            self.s_rnd = torch.randn(max_nodes, pooled_nodes)
            self.s_rnd.requires_grad = False
        elif pooling=='topk':
            self.pool = TopKPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='panpool':
            self.pool = PANPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='sagpool':
            self.pool = SAGPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='asapool':
            self.pool = ASAPooling(hidden_channels, ratio=pool_ratio)  
        elif pooling=='edgepool':
            self.pool = EdgePooling(hidden_channels)
        elif pooling=='kmis':
            self.pool = KMISPooling(hidden_channels, k=5, aggr_x='sum')
        elif pooling in ['graclus', 'comp-graclus']:
            pass
        elif pooling=='sparse-random':
            self.pool = RndSparse(pool_ratio, max_nodes)
        else:
            assert pooling==None
        
        # Post-pooling block
        self.conv_layers_post = torch.nn.ModuleList()
        for _ in range(num_layers_post):
            mlp = MLP([hidden_channels, hidden_channels, hidden_channels], act=activation, norm=None)
            if pooling in ['diffpool','mincut','dmon','dense-random']:
                self.conv_layers_post.append(DenseGINConv(nn=mlp, train_eps=False))                
            else:
                self.conv_layers_post.append(GINConv(nn=mlp, train_eps=False))

        # Readout
        self.mlp = MLP([hidden_channels, hidden_channels, hidden_channels//2, out_channels], 
                        act=activation,
                        norm=None,
                        dropout=0.5)


    def reset_parameters(self):
        for (name, module) in self._modules.items():
            try:
                module.reset_parameters()
            except AttributeError:
                if name != 'act':
                    for x in module:
                        x.reset_parameters()


    def forward(self, data):
        x = data.x    
        adj = data.edge_index
        batch = data.batch

        ### pre-pooling block
        if self.pooling=='panpool':
            M = adj
            for layer in self.conv_layers_pre:  
                x, M = layer(x, M)
                x = self.act(x)
        else:
            for layer in self.conv_layers_pre:  
                x = self.act(layer(x, adj))
    
        ### pooling block
        if self.pooling in ['diffpool','mincut','dmon','dense-random']:
            x, mask = to_dense_batch(x, batch)
            adj = to_dense_adj(adj, batch)
            if self.pooling=='diffpool':
                s = self.pool(x)
                x, adj, l1, l2 = dense_diff_pool(x, adj, s, mask)
                aux_loss = 0.1*l1 + 0.1*l2
            elif self.pooling=='mincut':
                s = self.pool(x)
                x, adj, l1, l2 = dense_mincut_pool(x, adj, s, mask)
                aux_loss = 0.5*l1 + l2
            elif self.pooling=='dmon':  
                _, x, adj, l1, l2, l3 = self.pool(x, adj, mask)
                aux_loss = 0.3*l1 + 0.3*l2 + 0.3*l3
            elif self.pooling=='dense-random':
                s = self.s_rnd[:x.size(1), :].unsqueeze(dim=0).expand(x.size(0), -1, -1).to(x.device)
                x, adj, _, _ = dense_diff_pool(x, adj, s, mask)
        elif self.pooling in ['topk', 'sagpool', 'sparse-random']:
            x, adj, _, batch, _, _ = self.pool(x, adj, edge_attr=None, batch=batch)
        elif self.pooling=='asapool':
            x, adj, _, batch, _ = self.pool(x, adj, batch=batch)
        elif self.pooling=='panpool':
            x, adj, _, batch, _, _ = self.pool(x, M, batch=batch)
        elif self.pooling=='edgepool':
            x, adj, batch, _ = self.pool(x, adj, batch=batch)
        elif self.pooling=='kmis':
            x, adj, _, batch, _, _ = self.pool(x, adj, None, batch=batch)
        elif self.pooling in ['graclus', 'comp-graclus']:
            data.x = x    
            if self.pooling == 'graclus':
                cluster = graclus(adj, num_nodes=data.x.size(0))
            else:
                complement = batched_negative_edges(edge_index=adj, batch=batch, force_undirected=True)
                cluster = graclus(complement, num_nodes=data.x.size(0))
            data = sum_pool(cluster, data)
            x = data.x    
            adj = data.edge_index
            batch = data.batch
        elif self.pooling==None:
            pass
        else:
            raise KeyError("unrecognized pooling method")
                
        ### post-pooling block
        for layer in self.conv_layers_post:  
            x = self.act(layer(x, adj))

        ### readout
        if self.pooling in ['diffpool','mincut','dmon','dense-random']:
            x = torch.sum(x, dim=1)
        else:
            x = global_add_pool(x, batch)
        x = self.mlp(x)
        
        if 'aux_loss' not in locals():
            aux_loss=0
        return F.log_softmax(x, dim=-1), aux_loss

In [21]:
def train(model, loader, optimizer):
    model.train()
    total_loss = 0
    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, aux_loss = model(data)
        loss = F.nll_loss(out, data.y) + aux_loss
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(loader.dataset)


In [22]:


@torch.no_grad()
def test(model, loader):
    model.eval()
    total_correct = 0
    total_loss = 0
    for data in loader:
        data = data.to(device)
        out, _ = model(data)
        loss = F.nll_loss(out, data.y)
        total_loss += float(loss) * data.num_graphs
        pred = out.argmax(dim=-1)
        total_correct += int((pred == data.y).sum())
    return total_correct / len(loader.dataset), total_loss / len(loader.dataset)

In [23]:
def log(**kwargs):
    def _map(value: Any) -> str:
        if isinstance(value, int) and not isinstance(value, bool):
            return f'{value:03d}'
        if isinstance(value, float):
            return f'{value:.4f}'
        return value

    print(', '.join(f'{key}: {_map(value)}' for key, value in kwargs.items()))

In [24]:
tot_acc = []
args.pooling = None
for r in range(args.runs):  
    print(args.pooling)
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

None
Epoch: 001, Loss: 0.7193, Train: 0.6325, Val: 0.6300, Test: 0.5300
Epoch: 002, Loss: 0.6026, Train: 0.7283, Val: 0.7133, Test: 0.7033
Epoch: 003, Loss: 0.5530, Train: 0.6221, Val: 0.6200, Test: 0.5300
Epoch: 004, Loss: 0.5325, Train: 0.8042, Val: 0.7667, Test: 0.7567
Epoch: 005, Loss: 0.4778, Train: 0.7833, Val: 0.7733, Test: 0.7900
Epoch: 006, Loss: 0.4676, Train: 0.6879, Val: 0.7333, Test: 0.7400
Epoch: 007, Loss: 0.4268, Train: 0.6471, Val: 0.6433, Test: 0.5400
Epoch: 008, Loss: 0.3969, Train: 0.6871, Val: 0.7233, Test: 0.7400
Epoch: 009, Loss: 0.3576, Train: 0.5958, Val: 0.5967, Test: 0.4900
Epoch: 010, Loss: 0.3727, Train: 0.8617, Val: 0.8467, Test: 0.8200
Epoch: 011, Loss: 0.3391, Train: 0.8429, Val: 0.8233, Test: 0.7867
Epoch: 012, Loss: 0.3276, Train: 0.8929, Val: 0.8867, Test: 0.8800
Epoch: 013, Loss: 0.3091, Train: 0.8008, Val: 0.8167, Test: 0.8567
Epoch: 014, Loss: 0.2854, Train: 0.8567, Val: 0.8467, Test: 0.7967
Epoch: 015, Loss: 0.2700, Train: 0.9217, Val: 0.9400, Tes

In [25]:
tot_acc = []
args.pooling = 'diffpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

diffpool
Epoch: 001, Loss: 1.4717, Train: 0.5188, Val: 0.5467, Test: 0.5167
Epoch: 002, Loss: 0.9326, Train: 0.7029, Val: 0.7467, Test: 0.6667
Epoch: 003, Loss: 0.8324, Train: 0.6967, Val: 0.7200, Test: 0.6933
Epoch: 004, Loss: 0.7807, Train: 0.6742, Val: 0.6967, Test: 0.6733
Epoch: 005, Loss: 0.7794, Train: 0.7571, Val: 0.7500, Test: 0.7167
Epoch: 006, Loss: 0.7395, Train: 0.5296, Val: 0.5533, Test: 0.5300
Epoch: 007, Loss: 0.7032, Train: 0.7733, Val: 0.7800, Test: 0.7667
Epoch: 008, Loss: 0.6860, Train: 0.7750, Val: 0.7933, Test: 0.7733
Epoch: 009, Loss: 0.6342, Train: 0.6854, Val: 0.6667, Test: 0.6933
Epoch: 010, Loss: 0.6242, Train: 0.7992, Val: 0.8300, Test: 0.8133
Epoch: 011, Loss: 0.6259, Train: 0.5383, Val: 0.5667, Test: 0.5500
Epoch: 012, Loss: 0.5869, Train: 0.8246, Val: 0.8167, Test: 0.8533
Epoch: 013, Loss: 0.5920, Train: 0.7096, Val: 0.7200, Test: 0.7133
Epoch: 014, Loss: 0.5574, Train: 0.8021, Val: 0.7933, Test: 0.7833
Epoch: 015, Loss: 0.5579, Train: 0.7808, Val: 0.7500,

In [26]:
tot_acc = []
args.pooling = 'mincut'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

mincut
Epoch: 001, Loss: 1.3456, Train: 0.6683, Val: 0.7133, Test: 0.6467
Epoch: 002, Loss: 1.2571, Train: 0.6883, Val: 0.7500, Test: 0.6700
Epoch: 003, Loss: 1.2319, Train: 0.6671, Val: 0.7233, Test: 0.6567
Epoch: 004, Loss: 1.1923, Train: 0.7471, Val: 0.8000, Test: 0.7500
Epoch: 005, Loss: 1.1746, Train: 0.7612, Val: 0.8133, Test: 0.7833
Epoch: 006, Loss: 1.1041, Train: 0.5896, Val: 0.5900, Test: 0.6133
Epoch: 007, Loss: 1.0735, Train: 0.7700, Val: 0.8100, Test: 0.8000
Epoch: 008, Loss: 1.0247, Train: 0.7479, Val: 0.7867, Test: 0.7567
Epoch: 009, Loss: 0.9955, Train: 0.7562, Val: 0.7667, Test: 0.7667
Epoch: 010, Loss: 0.9727, Train: 0.8096, Val: 0.8200, Test: 0.8200
Epoch: 011, Loss: 0.9352, Train: 0.8329, Val: 0.8667, Test: 0.8533
Epoch: 012, Loss: 0.9222, Train: 0.7000, Val: 0.6967, Test: 0.7000
Epoch: 013, Loss: 0.9029, Train: 0.7854, Val: 0.7933, Test: 0.7900
Epoch: 014, Loss: 0.9089, Train: 0.8538, Val: 0.8700, Test: 0.8600
Epoch: 015, Loss: 0.8691, Train: 0.8758, Val: 0.9033, T

In [27]:
tot_acc = []
args.pooling = 'dmon'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

dmon
Epoch: 001, Loss: 2.0819, Train: 0.6804, Val: 0.7167, Test: 0.6767
Epoch: 002, Loss: 2.0178, Train: 0.7079, Val: 0.7000, Test: 0.6867
Epoch: 003, Loss: 1.9866, Train: 0.5917, Val: 0.5933, Test: 0.5533
Epoch: 004, Loss: 1.9342, Train: 0.7467, Val: 0.7033, Test: 0.7067
Epoch: 005, Loss: 1.8908, Train: 0.7104, Val: 0.7167, Test: 0.7033
Epoch: 006, Loss: 1.8614, Train: 0.8004, Val: 0.7967, Test: 0.7600
Epoch: 007, Loss: 1.8344, Train: 0.6504, Val: 0.6633, Test: 0.6633
Epoch: 008, Loss: 1.8108, Train: 0.7188, Val: 0.7033, Test: 0.6933
Epoch: 009, Loss: 1.7972, Train: 0.7854, Val: 0.7800, Test: 0.7733
Epoch: 010, Loss: 1.7777, Train: 0.7950, Val: 0.7867, Test: 0.7867
Epoch: 011, Loss: 1.7669, Train: 0.8175, Val: 0.8200, Test: 0.8100
Epoch: 012, Loss: 1.7795, Train: 0.7617, Val: 0.7533, Test: 0.7533
Epoch: 013, Loss: 1.7434, Train: 0.7996, Val: 0.8033, Test: 0.7900
Epoch: 014, Loss: 1.7549, Train: 0.8529, Val: 0.8567, Test: 0.8567
Epoch: 015, Loss: 1.7523, Train: 0.8329, Val: 0.8267, Tes

In [28]:
tot_acc = []
args.pooling = 'edgepool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

edgepool
Epoch: 001, Loss: 0.7675, Train: 0.7150, Val: 0.7033, Test: 0.7167
Epoch: 002, Loss: 0.5985, Train: 0.8029, Val: 0.7567, Test: 0.7667
Epoch: 003, Loss: 0.4961, Train: 0.8525, Val: 0.8133, Test: 0.8233
Epoch: 004, Loss: 0.4324, Train: 0.8488, Val: 0.8333, Test: 0.8767
Epoch: 005, Loss: 0.3683, Train: 0.8400, Val: 0.8367, Test: 0.8367
Epoch: 006, Loss: 0.3411, Train: 0.8683, Val: 0.8467, Test: 0.8867
Epoch: 007, Loss: 0.3242, Train: 0.9125, Val: 0.8767, Test: 0.8800
Epoch: 008, Loss: 0.2928, Train: 0.9042, Val: 0.8967, Test: 0.9033
Epoch: 009, Loss: 0.2644, Train: 0.9125, Val: 0.8967, Test: 0.9233
Epoch: 010, Loss: 0.2529, Train: 0.9263, Val: 0.8867, Test: 0.8933
Epoch: 011, Loss: 0.2446, Train: 0.9300, Val: 0.9000, Test: 0.9100
Epoch: 012, Loss: 0.2594, Train: 0.9342, Val: 0.9167, Test: 0.9267
Epoch: 013, Loss: 0.2156, Train: 0.8975, Val: 0.8900, Test: 0.9333
Epoch: 014, Loss: 0.2129, Train: 0.8458, Val: 0.8933, Test: 0.8700
Epoch: 015, Loss: 0.2023, Train: 0.9267, Val: 0.9133,

In [29]:
tot_acc = []
args.pooling = 'graclus'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

graclus
Epoch: 001, Loss: 0.8587, Train: 0.5725, Val: 0.5167, Test: 0.5267
Epoch: 002, Loss: 0.6111, Train: 0.7217, Val: 0.7267, Test: 0.7600
Epoch: 003, Loss: 0.5578, Train: 0.7650, Val: 0.7267, Test: 0.7400
Epoch: 004, Loss: 0.4936, Train: 0.6833, Val: 0.6200, Test: 0.6467
Epoch: 005, Loss: 0.4261, Train: 0.8567, Val: 0.8800, Test: 0.8733
Epoch: 006, Loss: 0.3772, Train: 0.7646, Val: 0.7133, Test: 0.7467
Epoch: 007, Loss: 0.3534, Train: 0.7913, Val: 0.8100, Test: 0.8233
Epoch: 008, Loss: 0.3314, Train: 0.8771, Val: 0.8133, Test: 0.8833
Epoch: 009, Loss: 0.3215, Train: 0.8983, Val: 0.9133, Test: 0.9233
Epoch: 010, Loss: 0.3043, Train: 0.8954, Val: 0.8233, Test: 0.8833
Epoch: 011, Loss: 0.2973, Train: 0.8825, Val: 0.8933, Test: 0.9100
Epoch: 012, Loss: 0.2881, Train: 0.9171, Val: 0.9100, Test: 0.9367
Epoch: 013, Loss: 0.2472, Train: 0.9292, Val: 0.9100, Test: 0.9267
Epoch: 014, Loss: 0.2363, Train: 0.8883, Val: 0.8333, Test: 0.8967
Epoch: 015, Loss: 0.2263, Train: 0.9279, Val: 0.9400, 

In [30]:
tot_acc = []
args.pooling = 'kmis'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

kmis
Epoch: 001, Loss: 0.6769, Train: 0.7987, Val: 0.7700, Test: 0.7800
Epoch: 002, Loss: 0.4728, Train: 0.8263, Val: 0.8167, Test: 0.8067
Epoch: 003, Loss: 0.3597, Train: 0.7996, Val: 0.7600, Test: 0.7867
Epoch: 004, Loss: 0.2463, Train: 0.9446, Val: 0.9200, Test: 0.9300
Epoch: 005, Loss: 0.1901, Train: 0.9492, Val: 0.9467, Test: 0.9433
Epoch: 006, Loss: 0.1283, Train: 0.9717, Val: 0.9700, Test: 0.9600
Epoch: 007, Loss: 0.1067, Train: 0.9792, Val: 0.9767, Test: 0.9700
Epoch: 008, Loss: 0.0863, Train: 0.9775, Val: 0.9700, Test: 0.9800
Epoch: 009, Loss: 0.0778, Train: 0.9829, Val: 0.9700, Test: 0.9833
Epoch: 010, Loss: 0.0715, Train: 0.9892, Val: 0.9833, Test: 0.9867
Epoch: 011, Loss: 0.0577, Train: 0.9900, Val: 0.9867, Test: 0.9867
Epoch: 012, Loss: 0.0617, Train: 0.9838, Val: 0.9767, Test: 0.9933
Epoch: 013, Loss: 0.0448, Train: 0.9875, Val: 0.9800, Test: 0.9800
Epoch: 014, Loss: 0.0450, Train: 0.9896, Val: 0.9900, Test: 0.9833
Epoch: 015, Loss: 0.0334, Train: 0.9879, Val: 0.9867, Tes

In [31]:
tot_acc = []
args.pooling = 'topk'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

topk
Epoch: 001, Loss: 0.7051, Train: 0.5088, Val: 0.5000, Test: 0.4500
Epoch: 002, Loss: 0.6996, Train: 0.5333, Val: 0.5433, Test: 0.5133
Epoch: 003, Loss: 0.6905, Train: 0.5054, Val: 0.5033, Test: 0.4533
Epoch: 004, Loss: 0.6674, Train: 0.7662, Val: 0.7433, Test: 0.7500
Epoch: 005, Loss: 0.6909, Train: 0.5396, Val: 0.5633, Test: 0.5067
Epoch: 006, Loss: 0.6463, Train: 0.5088, Val: 0.5067, Test: 0.4567
Epoch: 007, Loss: 0.6540, Train: 0.7812, Val: 0.7400, Test: 0.7767
Epoch: 008, Loss: 0.6745, Train: 0.5054, Val: 0.5033, Test: 0.4533
Epoch: 009, Loss: 0.6891, Train: 0.7300, Val: 0.7233, Test: 0.7233
Epoch: 010, Loss: 0.6437, Train: 0.7804, Val: 0.7500, Test: 0.7700
Epoch: 011, Loss: 0.7038, Train: 0.5092, Val: 0.5133, Test: 0.4633
Epoch: 012, Loss: 0.7033, Train: 0.4813, Val: 0.5067, Test: 0.5167
Epoch: 013, Loss: 0.6990, Train: 0.4988, Val: 0.5067, Test: 0.5367
Epoch: 014, Loss: 0.6940, Train: 0.5587, Val: 0.5700, Test: 0.5333
Epoch: 015, Loss: 0.6947, Train: 0.5200, Val: 0.5200, Tes

In [32]:
tot_acc = []
args.pooling = 'panpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

panpool


c:\Users\USER\anaconda3\envs\un\lib\site-packages\torch_sparse\matmul.py:97: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:56.)
  C = torch.sparse.mm(A, B)


Epoch: 001, Loss: 0.7304, Train: 0.4863, Val: 0.4967, Test: 0.4833
Epoch: 002, Loss: 0.7047, Train: 0.5258, Val: 0.5400, Test: 0.5033
Epoch: 003, Loss: 0.7032, Train: 0.5029, Val: 0.4767, Test: 0.5200
Epoch: 004, Loss: 0.7008, Train: 0.5483, Val: 0.5367, Test: 0.5600
Epoch: 005, Loss: 0.6961, Train: 0.5429, Val: 0.5533, Test: 0.5233
Epoch: 006, Loss: 0.6952, Train: 0.5475, Val: 0.5433, Test: 0.5433
Epoch: 007, Loss: 0.6918, Train: 0.5421, Val: 0.5400, Test: 0.5367
Epoch: 008, Loss: 0.6954, Train: 0.5504, Val: 0.5367, Test: 0.5567
Epoch: 009, Loss: 0.6965, Train: 0.5554, Val: 0.5200, Test: 0.5533
Epoch: 010, Loss: 0.6923, Train: 0.5212, Val: 0.5233, Test: 0.5267
Epoch: 011, Loss: 0.6965, Train: 0.5396, Val: 0.5333, Test: 0.5367
Epoch: 012, Loss: 0.6940, Train: 0.5567, Val: 0.5300, Test: 0.5667
Epoch: 013, Loss: 0.6937, Train: 0.5463, Val: 0.5167, Test: 0.5100
Epoch: 014, Loss: 0.6955, Train: 0.5088, Val: 0.5000, Test: 0.5167
Epoch: 015, Loss: 0.6962, Train: 0.5500, Val: 0.5400, Test: 0.

In [33]:
tot_acc = []
args.pooling = 'asapool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

asapool
Epoch: 001, Loss: 0.7508, Train: 0.5071, Val: 0.5133, Test: 0.5800
Epoch: 002, Loss: 0.7031, Train: 0.5212, Val: 0.5400, Test: 0.5400
Epoch: 003, Loss: 0.6992, Train: 0.5225, Val: 0.5200, Test: 0.5000
Epoch: 004, Loss: 0.7000, Train: 0.5825, Val: 0.5533, Test: 0.5733
Epoch: 005, Loss: 0.6940, Train: 0.5437, Val: 0.5233, Test: 0.5367
Epoch: 006, Loss: 0.6956, Train: 0.5342, Val: 0.5167, Test: 0.5233
Epoch: 007, Loss: 0.6591, Train: 0.6704, Val: 0.6533, Test: 0.6867
Epoch: 008, Loss: 0.6463, Train: 0.6775, Val: 0.6667, Test: 0.6933
Epoch: 009, Loss: 0.5717, Train: 0.7863, Val: 0.7833, Test: 0.7833
Epoch: 010, Loss: 0.3676, Train: 0.8008, Val: 0.8033, Test: 0.7900
Epoch: 011, Loss: 0.5334, Train: 0.6629, Val: 0.6933, Test: 0.6833
Epoch: 012, Loss: 0.4098, Train: 0.8842, Val: 0.8900, Test: 0.8767
Epoch: 013, Loss: 0.3391, Train: 0.8896, Val: 0.8933, Test: 0.8800
Epoch: 014, Loss: 0.3488, Train: 0.8617, Val: 0.8533, Test: 0.8467
Epoch: 015, Loss: 0.3345, Train: 0.8946, Val: 0.9033, 

In [34]:
tot_acc = []
args.pooling = 'sagpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

sagpool
Epoch: 001, Loss: 0.6997, Train: 0.5062, Val: 0.4700, Test: 0.4800
Epoch: 002, Loss: 0.6989, Train: 0.5021, Val: 0.5167, Test: 0.4833
Epoch: 003, Loss: 0.7013, Train: 0.5067, Val: 0.4600, Test: 0.4800
Epoch: 004, Loss: 0.6976, Train: 0.5050, Val: 0.5000, Test: 0.4700
Epoch: 005, Loss: 0.6945, Train: 0.4871, Val: 0.4367, Test: 0.4767
Epoch: 006, Loss: 0.6939, Train: 0.5096, Val: 0.5067, Test: 0.5767
Epoch: 007, Loss: 0.6925, Train: 0.6512, Val: 0.6600, Test: 0.6333
Epoch: 008, Loss: 0.6528, Train: 0.6558, Val: 0.6467, Test: 0.6733
Epoch: 009, Loss: 0.5654, Train: 0.6887, Val: 0.7167, Test: 0.7167
Epoch: 010, Loss: 0.5096, Train: 0.8071, Val: 0.7733, Test: 0.7967
Epoch: 011, Loss: 0.4519, Train: 0.8129, Val: 0.8267, Test: 0.7933
Epoch: 012, Loss: 0.4315, Train: 0.8287, Val: 0.8233, Test: 0.8367
Epoch: 013, Loss: 0.4082, Train: 0.8263, Val: 0.8367, Test: 0.8433
Epoch: 014, Loss: 0.4503, Train: 0.8279, Val: 0.8500, Test: 0.8633
Epoch: 015, Loss: 0.4161, Train: 0.8033, Val: 0.8100, 

In [35]:
tot_acc = []
args.pooling = 'dense-random'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

dense-random
Epoch: 001, Loss: 1.8676, Train: 0.6613, Val: 0.6633, Test: 0.6600
Epoch: 002, Loss: 0.7858, Train: 0.7283, Val: 0.7500, Test: 0.7400
Epoch: 003, Loss: 0.7029, Train: 0.7462, Val: 0.7567, Test: 0.7733
Epoch: 004, Loss: 0.6056, Train: 0.6913, Val: 0.7100, Test: 0.6933
Epoch: 005, Loss: 0.5591, Train: 0.5600, Val: 0.5800, Test: 0.5567
Epoch: 006, Loss: 0.5409, Train: 0.7642, Val: 0.7500, Test: 0.7633
Epoch: 007, Loss: 0.5011, Train: 0.8142, Val: 0.8100, Test: 0.8300
Epoch: 008, Loss: 0.4773, Train: 0.8233, Val: 0.8233, Test: 0.8433
Epoch: 009, Loss: 0.4199, Train: 0.8017, Val: 0.8133, Test: 0.8433
Epoch: 010, Loss: 0.4124, Train: 0.8283, Val: 0.8133, Test: 0.8433
Epoch: 011, Loss: 0.3978, Train: 0.8733, Val: 0.8700, Test: 0.8600
Epoch: 012, Loss: 0.3996, Train: 0.7087, Val: 0.7167, Test: 0.7100
Epoch: 013, Loss: 0.3868, Train: 0.8467, Val: 0.8567, Test: 0.8500
Epoch: 014, Loss: 0.3518, Train: 0.8413, Val: 0.8467, Test: 0.8600
Epoch: 015, Loss: 0.3800, Train: 0.8871, Val: 0.8

In [36]:
tot_acc = []
args.pooling = 'comp-graclus'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GIN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

comp-graclus
Epoch: 001, Loss: 0.7728, Train: 0.5038, Val: 0.5367, Test: 0.5300
Epoch: 002, Loss: 0.6387, Train: 0.7362, Val: 0.7267, Test: 0.7100
Epoch: 003, Loss: 0.5851, Train: 0.6558, Val: 0.6367, Test: 0.6567
Epoch: 004, Loss: 0.5485, Train: 0.7179, Val: 0.7133, Test: 0.7000
Epoch: 005, Loss: 0.5180, Train: 0.7342, Val: 0.7033, Test: 0.7200
Epoch: 006, Loss: 0.4851, Train: 0.6333, Val: 0.6067, Test: 0.6267
Epoch: 007, Loss: 0.4877, Train: 0.8100, Val: 0.7833, Test: 0.7767
Epoch: 008, Loss: 0.4687, Train: 0.6929, Val: 0.7067, Test: 0.6900
Epoch: 009, Loss: 0.4487, Train: 0.6275, Val: 0.6133, Test: 0.6133
Epoch: 010, Loss: 0.4358, Train: 0.7771, Val: 0.7633, Test: 0.7400
Epoch: 011, Loss: 0.4503, Train: 0.8004, Val: 0.8333, Test: 0.7733
Epoch: 012, Loss: 0.4399, Train: 0.6583, Val: 0.6467, Test: 0.6300
Epoch: 013, Loss: 0.4200, Train: 0.8346, Val: 0.8533, Test: 0.8233
Epoch: 014, Loss: 0.4300, Train: 0.6871, Val: 0.7200, Test: 0.7033
Epoch: 015, Loss: 0.4133, Train: 0.8538, Val: 0.8

KeyboardInterrupt: 

In [74]:
class GCN_Pool_Net(torch.nn.Module):
    def __init__(self, 
                 in_channels,           # Size of node features
                 out_channels,          # Number of classes
                 num_layers_pre=1,      # Number of GIN layers before pooling
                 num_layers_post=1,     # Number of GIN layers after pooling
                 hidden_channels=64,    # Dimensionality of node embeddings
                 norm=True,             # Normalise Layers in the GIN MLP
                 activation='ELU',      # Activation of the MLP in GIN 
                 average_nodes=None,    # Needed for dense pooling methods
                 max_nodes=None,        # Needed for random pool
                 pooling=None,          # Pooling method
                 pool_ratio=0.1,        # Ratio = nodes_after_pool/nodes_before_pool
                 ):
        super(GCN_Pool_Net, self).__init__()
        
        self.num_layers_pre = num_layers_pre
        self.num_layers_post = num_layers_post
        self.hidden_channels = hidden_channels
        self.act = activation_resolver(activation)
        self.pooling = pooling
        self.pool_ratio = pool_ratio
  
        # Pre-pooling block            
        self.conv_layers_pre = torch.nn.ModuleList()
        if pooling=='panpool':
            for _ in range(num_layers_pre):
                self.conv_layers_pre.append(PANConv(in_channels, hidden_channels, filter_size=2))
                in_channels = hidden_channels
        else:
            for _ in range(num_layers_pre):
                self.conv_layers_pre.append(GCNConv(in_channels, hidden_channels))
                in_channels = hidden_channels
                        
        # Pooling block
        pooled_nodes = ceil(pool_ratio * average_nodes)
        if pooling in ['diffpool','mincut']:
            self.pool = Linear(hidden_channels, pooled_nodes)
        elif pooling=='dmon':
            self.pool = DMoNPooling(hidden_channels, pooled_nodes)
        elif pooling=='dense-random':
            self.s_rnd = torch.randn(max_nodes, pooled_nodes)
            self.s_rnd.requires_grad = False
        elif pooling=='topk':
            self.pool = TopKPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='panpool':
            self.pool = PANPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='sagpool':
            self.pool = SAGPooling(hidden_channels, ratio=pool_ratio)
        elif pooling=='asapool':
            self.pool = ASAPooling(hidden_channels, ratio=pool_ratio)  
        elif pooling=='edgepool':
            self.pool = EdgePooling(hidden_channels)
        elif pooling=='kmis':
            self.pool = KMISPooling(hidden_channels, k=5, aggr_x='sum')
        elif pooling in ['graclus', 'comp-graclus']:
            pass
        elif pooling=='sparse-random':
            self.pool = RndSparse(pool_ratio, max_nodes)
        else:
            assert pooling==None
        
        # Post-pooling block
        self.conv_layers_post = torch.nn.ModuleList()
        for _ in range(num_layers_post):
            mlp = MLP([hidden_channels, hidden_channels, hidden_channels], act=activation, norm=None)
            if pooling in ['diffpool','mincut','dmon','dense-random']:
                self.conv_layers_post.append(DenseGCNConv(hidden_channels, hidden_channels, mlp))                 
            else:
                (self.conv_layers_post.append(GCNConv(hidden_channels, hidden_channels))
)

        # Readout
        self.mlp = MLP([hidden_channels, hidden_channels, hidden_channels//2, out_channels], 
                        act=activation,
                        norm=None,
                        dropout=0.5)


    def reset_parameters(self):
        for (name, module) in self._modules.items():
            try:
                module.reset_parameters()
            except AttributeError:
                if name != 'act':
                    for x in module:
                        x.reset_parameters()


    def forward(self, data):
        x = data.x    
        adj = data.edge_index
        batch = data.batch

        ### pre-pooling block
        if self.pooling=='panpool':
            M = adj
            for layer in self.conv_layers_pre:  
                x, M = layer(x, M)
                x = self.act(x)
        else:
            for layer in self.conv_layers_pre:  
                x = self.act(layer(x, adj))
    
        ### pooling block
        if self.pooling in ['diffpool','mincut','dmon','dense-random']:
            x, mask = to_dense_batch(x, batch)
            adj = to_dense_adj(adj, batch)
            if self.pooling=='diffpool':
                s = self.pool(x)
                x, adj, l1, l2 = dense_diff_pool(x, adj, s, mask)
                aux_loss = 0.1*l1 + 0.1*l2
            elif self.pooling=='mincut':
                s = self.pool(x)
                x, adj, l1, l2 = dense_mincut_pool(x, adj, s, mask)
                aux_loss = 0.5*l1 + l2
            elif self.pooling=='dmon':  
                _, x, adj, l1, l2, l3 = self.pool(x, adj, mask)
                aux_loss = 0.3*l1 + 0.3*l2 + 0.3*l3
            elif self.pooling=='dense-random':
                s = self.s_rnd[:x.size(1), :].unsqueeze(dim=0).expand(x.size(0), -1, -1).to(x.device)
                x, adj, _, _ = dense_diff_pool(x, adj, s, mask)
        elif self.pooling in ['topk', 'sagpool', 'sparse-random']:
            x, adj, _, batch, _, _ = self.pool(x, adj, edge_attr=None, batch=batch)
        elif self.pooling=='asapool':
            x, adj, _, batch, _ = self.pool(x, adj, batch=batch)
        elif self.pooling=='panpool':
            x, adj, _, batch, _, _ = self.pool(x, M, batch=batch)
        elif self.pooling=='edgepool':
            x, adj, batch, _ = self.pool(x, adj, batch=batch)
        elif self.pooling=='kmis':
            x, adj, _, batch, _, _ = self.pool(x, adj, None, batch=batch)
        elif self.pooling in ['graclus', 'comp-graclus']:
            data.x = x    
            if self.pooling == 'graclus':
                cluster = graclus(adj, num_nodes=data.x.size(0))
            else:
                complement = batched_negative_edges(edge_index=adj, batch=batch, force_undirected=True)
                cluster = graclus(complement, num_nodes=data.x.size(0))
            data = sum_pool(cluster, data)
            x = data.x    
            adj = data.edge_index
            batch = data.batch
        elif self.pooling==None:
            pass
        else:
            raise KeyError("unrecognized pooling method")
                
        ### post-pooling block
        for layer in self.conv_layers_post:  
            x = self.act(layer(x, adj))

        ### readout
        if self.pooling in ['diffpool','mincut','dmon','dense-random']:
            x = torch.sum(x, dim=1)
        else:
            x = global_add_pool(x, batch)
        x = self.mlp(x)
        
        if 'aux_loss' not in locals():
            aux_loss=0
        return F.log_softmax(x, dim=-1), aux_loss

In [61]:
tot_acc = []
args.pooling = None
for r in range(args.runs):  
    print(args.pooling)
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

None
Epoch: 001, Loss: 0.9501, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 002, Loss: 0.7609, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 003, Loss: 0.7361, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 004, Loss: 0.7309, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 005, Loss: 0.7173, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 006, Loss: 0.7073, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 007, Loss: 0.7122, Train: 0.4983, Val: 0.5133, Test: 0.5000
Epoch: 008, Loss: 0.7136, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 009, Loss: 0.7067, Train: 0.5012, Val: 0.4967, Test: 0.5000
Epoch: 010, Loss: 0.7078, Train: 0.5017, Val: 0.4867, Test: 0.5000
Epoch: 011, Loss: 0.7078, Train: 0.4983, Val: 0.5133, Test: 0.5000
Epoch: 012, Loss: 0.7079, Train: 0.4992, Val: 0.5067, Test: 0.5000
Epoch: 013, Loss: 0.7054, Train: 0.4983, Val: 0.5133, Test: 0.5000
Epoch: 014, Loss: 0.7015, Train: 0.5083, Val: 0.4767, Test: 0.5267
Epoch: 015, Loss: 0.7071, Train: 0.4983, Val: 0.5133, Tes

In [75]:
tot_acc = []
args.pooling = 'diffpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


diffpool
Epoch: 001, Loss: 1.0025, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 002, Loss: 0.9284, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 003, Loss: 0.8966, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 004, Loss: 0.8822, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 005, Loss: 0.8838, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 006, Loss: 0.8705, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 007, Loss: 0.8749, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 008, Loss: 0.8695, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 009, Loss: 0.8660, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 010, Loss: 0.8610, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 011, Loss: 0.8550, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 012, Loss: 0.8550, Train: 0.5008, Val: 0.5167, Test: 0.4767
Epoch: 013, Loss: 0.8501, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 014, Loss: 0.8474, Train: 0.4992, Val: 0.4833, Test: 0.5233
Epoch: 015, Loss: 0.8455, Train: 0.5008, Val: 0.5167,

In [76]:

tot_acc = []
args.pooling = 'mincut'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


mincut
Epoch: 001, Loss: 1.4766, Train: 0.5008, Val: 0.5300, Test: 0.4633
Epoch: 002, Loss: 1.3965, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 003, Loss: 1.3680, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 004, Loss: 1.3560, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 005, Loss: 1.3486, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 006, Loss: 1.3495, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 007, Loss: 1.3473, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 008, Loss: 1.3457, Train: 0.5046, Val: 0.5100, Test: 0.4533
Epoch: 009, Loss: 1.3475, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 010, Loss: 1.3414, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 011, Loss: 1.3408, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 012, Loss: 1.3464, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 013, Loss: 1.3441, Train: 0.4988, Val: 0.4733, Test: 0.5367
Epoch: 014, Loss: 1.3429, Train: 0.5012, Val: 0.5267, Test: 0.4633
Epoch: 015, Loss: 1.3361, Train: 0.4988, Val: 0.4733, T

In [77]:

tot_acc = []
args.pooling = 'dmon'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


dmon
Epoch: 001, Loss: 2.1830, Train: 0.5008, Val: 0.4800, Test: 0.5133
Epoch: 002, Loss: 2.1152, Train: 0.5012, Val: 0.4767, Test: 0.5133
Epoch: 003, Loss: 2.1136, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 004, Loss: 2.1099, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 005, Loss: 2.1022, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 006, Loss: 2.0942, Train: 0.5079, Val: 0.4767, Test: 0.4933
Epoch: 007, Loss: 2.0988, Train: 0.4979, Val: 0.5267, Test: 0.4867
Epoch: 008, Loss: 2.0978, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 009, Loss: 2.0914, Train: 0.5008, Val: 0.4800, Test: 0.5133
Epoch: 010, Loss: 2.1034, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 011, Loss: 2.0958, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 012, Loss: 2.0977, Train: 0.5083, Val: 0.4733, Test: 0.4833
Epoch: 013, Loss: 2.0926, Train: 0.5008, Val: 0.4800, Test: 0.5133
Epoch: 014, Loss: 2.1013, Train: 0.4992, Val: 0.5200, Test: 0.4867
Epoch: 015, Loss: 2.0950, Train: 0.5083, Val: 0.4733, Tes

In [78]:

tot_acc = []
args.pooling = 'edgepool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


edgepool
Epoch: 001, Loss: 0.7993, Train: 0.4938, Val: 0.5200, Test: 0.4867
Epoch: 002, Loss: 0.7306, Train: 0.4908, Val: 0.4733, Test: 0.4900
Epoch: 003, Loss: 0.7193, Train: 0.4946, Val: 0.5300, Test: 0.5133
Epoch: 004, Loss: 0.7123, Train: 0.5058, Val: 0.4700, Test: 0.4800
Epoch: 005, Loss: 0.7081, Train: 0.4946, Val: 0.5300, Test: 0.5133
Epoch: 006, Loss: 0.7075, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 007, Loss: 0.7036, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 008, Loss: 0.7084, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 009, Loss: 0.7092, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 010, Loss: 0.7080, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 011, Loss: 0.7046, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 012, Loss: 0.7061, Train: 0.4946, Val: 0.5300, Test: 0.5133
Epoch: 013, Loss: 0.7036, Train: 0.4908, Val: 0.5033, Test: 0.4733
Epoch: 014, Loss: 0.7004, Train: 0.5054, Val: 0.4700, Test: 0.4867
Epoch: 015, Loss: 0.6990, Train: 0.5054, Val: 0.4700,

In [79]:
tot_acc = []
args.pooling = 'graclus'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


graclus
Epoch: 001, Loss: 0.8851, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 002, Loss: 0.7244, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 003, Loss: 0.7228, Train: 0.5054, Val: 0.4933, Test: 0.4900
Epoch: 004, Loss: 0.7118, Train: 0.4988, Val: 0.5200, Test: 0.4900
Epoch: 005, Loss: 0.7088, Train: 0.5008, Val: 0.4800, Test: 0.5133
Epoch: 006, Loss: 0.7048, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 007, Loss: 0.7087, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 008, Loss: 0.7082, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 009, Loss: 0.7088, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 010, Loss: 0.7042, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 011, Loss: 0.7043, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 012, Loss: 0.7042, Train: 0.5008, Val: 0.5167, Test: 0.4833
Epoch: 013, Loss: 0.7078, Train: 0.5012, Val: 0.4800, Test: 0.5100
Epoch: 014, Loss: 0.7035, Train: 0.4988, Val: 0.5200, Test: 0.4900
Epoch: 015, Loss: 0.7032, Train: 0.4988, Val: 0.5200, 

In [80]:
tot_acc = []
args.pooling = 'kmis'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


kmis
Epoch: 001, Loss: 0.7271, Train: 0.4992, Val: 0.4967, Test: 0.5100
Epoch: 002, Loss: 0.7152, Train: 0.4942, Val: 0.5033, Test: 0.5433
Epoch: 003, Loss: 0.7057, Train: 0.5058, Val: 0.4967, Test: 0.4567
Epoch: 004, Loss: 0.7050, Train: 0.5058, Val: 0.4967, Test: 0.4567
Epoch: 005, Loss: 0.7008, Train: 0.5017, Val: 0.5067, Test: 0.4600
Epoch: 006, Loss: 0.7071, Train: 0.4829, Val: 0.5167, Test: 0.5533
Epoch: 007, Loss: 0.7002, Train: 0.5004, Val: 0.5000, Test: 0.5600
Epoch: 008, Loss: 0.7004, Train: 0.4942, Val: 0.5033, Test: 0.5433
Epoch: 009, Loss: 0.6998, Train: 0.5058, Val: 0.4967, Test: 0.4567
Epoch: 010, Loss: 0.6999, Train: 0.4942, Val: 0.5033, Test: 0.5433
Epoch: 011, Loss: 0.6974, Train: 0.5058, Val: 0.4967, Test: 0.4567
Epoch: 012, Loss: 0.7001, Train: 0.5058, Val: 0.4967, Test: 0.4567
Epoch: 013, Loss: 0.6986, Train: 0.4942, Val: 0.5033, Test: 0.5433
Epoch: 014, Loss: 0.6998, Train: 0.4942, Val: 0.5033, Test: 0.5433
Epoch: 015, Loss: 0.6965, Train: 0.4942, Val: 0.5033, Tes

In [81]:
tot_acc = []
args.pooling = 'topk'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


topk
Epoch: 001, Loss: 0.7006, Train: 0.4992, Val: 0.5333, Test: 0.4733
Epoch: 002, Loss: 0.6938, Train: 0.4979, Val: 0.5400, Test: 0.5100
Epoch: 003, Loss: 0.6946, Train: 0.4988, Val: 0.5367, Test: 0.4733
Epoch: 004, Loss: 0.6941, Train: 0.5142, Val: 0.5100, Test: 0.5033
Epoch: 005, Loss: 0.6934, Train: 0.4988, Val: 0.5367, Test: 0.4733
Epoch: 006, Loss: 0.6940, Train: 0.5225, Val: 0.4900, Test: 0.5233
Epoch: 007, Loss: 0.6931, Train: 0.4963, Val: 0.4900, Test: 0.5000
Epoch: 008, Loss: 0.6950, Train: 0.4913, Val: 0.5333, Test: 0.5033
Epoch: 009, Loss: 0.6955, Train: 0.5046, Val: 0.4967, Test: 0.5033
Epoch: 010, Loss: 0.6954, Train: 0.5050, Val: 0.4967, Test: 0.5000
Epoch: 011, Loss: 0.6950, Train: 0.5192, Val: 0.5133, Test: 0.5100
Epoch: 012, Loss: 0.6944, Train: 0.4913, Val: 0.5333, Test: 0.5033
Epoch: 013, Loss: 0.6937, Train: 0.5225, Val: 0.4900, Test: 0.5233
Epoch: 014, Loss: 0.6927, Train: 0.5200, Val: 0.5233, Test: 0.5267
Epoch: 015, Loss: 0.6940, Train: 0.5100, Val: 0.5100, Tes

In [82]:
tot_acc = []
args.pooling = 'panpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


panpool
Epoch: 001, Loss: 0.7029, Train: 0.4996, Val: 0.4967, Test: 0.5167
Epoch: 002, Loss: 0.6946, Train: 0.5017, Val: 0.5067, Test: 0.5267
Epoch: 003, Loss: 0.6920, Train: 0.5004, Val: 0.4967, Test: 0.5133
Epoch: 004, Loss: 0.6903, Train: 0.5667, Val: 0.5567, Test: 0.5800
Epoch: 005, Loss: 0.6924, Train: 0.5000, Val: 0.5000, Test: 0.5167
Epoch: 006, Loss: 0.6908, Train: 0.5621, Val: 0.5433, Test: 0.6133
Epoch: 007, Loss: 0.6942, Train: 0.5621, Val: 0.5567, Test: 0.5567
Epoch: 008, Loss: 0.6865, Train: 0.5513, Val: 0.5333, Test: 0.5233
Epoch: 009, Loss: 0.6906, Train: 0.5613, Val: 0.5667, Test: 0.5800
Epoch: 010, Loss: 0.6863, Train: 0.5413, Val: 0.5200, Test: 0.5167
Epoch: 011, Loss: 0.6892, Train: 0.5046, Val: 0.5067, Test: 0.4867
Epoch: 012, Loss: 0.6893, Train: 0.5600, Val: 0.5367, Test: 0.5533
Epoch: 013, Loss: 0.6892, Train: 0.5658, Val: 0.5533, Test: 0.6033
Epoch: 014, Loss: 0.6920, Train: 0.5179, Val: 0.5200, Test: 0.5633
Epoch: 015, Loss: 0.6890, Train: 0.5408, Val: 0.5267, 

In [83]:
tot_acc = []
args.pooling = 'asapool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


asapool
Epoch: 001, Loss: 0.6986, Train: 0.4858, Val: 0.4800, Test: 0.5300
Epoch: 002, Loss: 0.6982, Train: 0.4967, Val: 0.5133, Test: 0.5133
Epoch: 003, Loss: 0.6979, Train: 0.5154, Val: 0.5167, Test: 0.5700
Epoch: 004, Loss: 0.6957, Train: 0.4950, Val: 0.4933, Test: 0.5467
Epoch: 005, Loss: 0.6938, Train: 0.4983, Val: 0.5067, Test: 0.5100
Epoch: 006, Loss: 0.6965, Train: 0.4938, Val: 0.5267, Test: 0.5367
Epoch: 007, Loss: 0.6946, Train: 0.5021, Val: 0.4933, Test: 0.4900
Epoch: 008, Loss: 0.6958, Train: 0.5133, Val: 0.5300, Test: 0.5467
Epoch: 009, Loss: 0.6935, Train: 0.5167, Val: 0.5167, Test: 0.5533
Epoch: 010, Loss: 0.6951, Train: 0.5283, Val: 0.4900, Test: 0.5367
Epoch: 011, Loss: 0.6941, Train: 0.5033, Val: 0.5267, Test: 0.5500
Epoch: 012, Loss: 0.6949, Train: 0.4950, Val: 0.5133, Test: 0.4967
Epoch: 013, Loss: 0.6946, Train: 0.4963, Val: 0.5000, Test: 0.4700
Epoch: 014, Loss: 0.6938, Train: 0.4942, Val: 0.5033, Test: 0.5333
Epoch: 015, Loss: 0.6933, Train: 0.4846, Val: 0.4933, 

In [84]:
tot_acc = []
args.pooling = 'sagpool'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


sagpool
Epoch: 001, Loss: 0.6960, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 002, Loss: 0.6951, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 003, Loss: 0.6962, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 004, Loss: 0.6942, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 005, Loss: 0.6939, Train: 0.5150, Val: 0.5300, Test: 0.4233
Epoch: 006, Loss: 0.6932, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 007, Loss: 0.6924, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 008, Loss: 0.6945, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 009, Loss: 0.6944, Train: 0.5138, Val: 0.5100, Test: 0.4400
Epoch: 010, Loss: 0.6940, Train: 0.5021, Val: 0.5167, Test: 0.4700
Epoch: 011, Loss: 0.6938, Train: 0.5025, Val: 0.5100, Test: 0.5167
Epoch: 012, Loss: 0.6937, Train: 0.4975, Val: 0.4767, Test: 0.5400
Epoch: 013, Loss: 0.6943, Train: 0.5012, Val: 0.5200, Test: 0.4700
Epoch: 014, Loss: 0.6942, Train: 0.5025, Val: 0.5100, Test: 0.5233
Epoch: 015, Loss: 0.6947, Train: 0.5146, Val: 0.5267, 

In [85]:
tot_acc = []
args.pooling = 'dense-random'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")


dense-random
Epoch: 001, Loss: 0.8809, Train: 0.4958, Val: 0.5200, Test: 0.5133
Epoch: 002, Loss: 0.7698, Train: 0.4958, Val: 0.5200, Test: 0.5133
Epoch: 003, Loss: 0.7363, Train: 0.4958, Val: 0.5200, Test: 0.5133
Epoch: 004, Loss: 0.7252, Train: 0.5008, Val: 0.5033, Test: 0.5033
Epoch: 005, Loss: 0.7157, Train: 0.5042, Val: 0.4800, Test: 0.4867
Epoch: 006, Loss: 0.7102, Train: 0.4958, Val: 0.5200, Test: 0.5133
Epoch: 007, Loss: 0.7086, Train: 0.5042, Val: 0.4800, Test: 0.4867
Epoch: 008, Loss: 0.7063, Train: 0.5033, Val: 0.5133, Test: 0.4600
Epoch: 009, Loss: 0.7053, Train: 0.5042, Val: 0.4800, Test: 0.4867
Epoch: 010, Loss: 0.7044, Train: 0.4958, Val: 0.5200, Test: 0.5133
Epoch: 011, Loss: 0.7109, Train: 0.5108, Val: 0.5000, Test: 0.4767
Epoch: 012, Loss: 0.7036, Train: 0.5088, Val: 0.4700, Test: 0.4667
Epoch: 013, Loss: 0.7057, Train: 0.5029, Val: 0.4867, Test: 0.4900
Epoch: 014, Loss: 0.7035, Train: 0.5012, Val: 0.4933, Test: 0.4967
Epoch: 015, Loss: 0.7034, Train: 0.5125, Val: 0.5

In [86]:
tot_acc = []
args.pooling = 'comp-graclus'
print(args.pooling)
for r in range(args.runs):  
    # Random shuffle the data
    rnd_idx = rng.permutation(len(dataset))
    dataset = dataset[list(rnd_idx)]
    
    train_dataset = dataset[len(dataset) // 5:]
    val_dataset = dataset[:len(dataset) // 10]
    test_dataset = dataset[len(dataset) // 10:len(dataset) // 5]
    
    train_loader = DataLoader(train_dataset, args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, args.batch_size)
    test_loader = DataLoader(test_dataset, args.batch_size)
    
    # Init the GNN
    net_model = GCN_Pool_Net(in_channels=train_dataset.num_features, 
                        out_channels=train_dataset.num_classes,
                        num_layers_pre=args.num_layers_pre,
                        num_layers_post=args.num_layers_post,
                        hidden_channels=args.hidden_channels,
                        average_nodes=avg_nodes,
                        pooling=args.pooling,
                        pool_ratio=args.pool_ratio,
                        max_nodes=max_nodes
                        ).to(device)
    opt = torch.optim.Adam(net_model.parameters(), lr=args.lr) 
    best_val=np.inf
    best_test=0
    for epoch in range(1, args.epochs + 1):
        loss = train(net_model, train_loader, opt)
        train_acc, _ = test(net_model, train_loader)
        val_acc, val_loss = test(net_model, val_loader)
        test_acc, _ = test(net_model, test_loader)
        if val_loss < best_val:
            best_val = val_loss
            best_test = test_acc
        log(Epoch=epoch, Loss=loss, Train=train_acc, Val=val_acc, Test=test_acc)
        
    tot_acc.append(best_test)
    print(f"### Run {r:d} - val loss: {best_val:.3f}, test acc: {best_test:.3f}")
    
print("Accuracies in each run: ", tot_acc)    
print(f"test acc - mean: {np.mean(tot_acc):.3f}, std: {np.std(tot_acc):.3f}")

comp-graclus
Epoch: 001, Loss: 0.7440, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 002, Loss: 0.7172, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 003, Loss: 0.7097, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 004, Loss: 0.7085, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 005, Loss: 0.7039, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 006, Loss: 0.7018, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 007, Loss: 0.7027, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 008, Loss: 0.7076, Train: 0.5029, Val: 0.4900, Test: 0.4867
Epoch: 009, Loss: 0.7050, Train: 0.5012, Val: 0.4900, Test: 0.4900
Epoch: 010, Loss: 0.7013, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 011, Loss: 0.7041, Train: 0.4971, Val: 0.5100, Test: 0.5133
Epoch: 012, Loss: 0.7003, Train: 0.5054, Val: 0.4733, Test: 0.4833
Epoch: 013, Loss: 0.6999, Train: 0.4958, Val: 0.5167, Test: 0.5133
Epoch: 014, Loss: 0.7029, Train: 0.5042, Val: 0.4767, Test: 0.4833
Epoch: 015, Loss: 0.7035, Train: 0.4963, Val: 0.5